# Catnip

## Setup

### Colab

In [ ]:
# --- Scripts for Colab ---
# Clone repo
!git clone -b yolo-finetune --recurse-submodules https://github.com/rifusaki/catnip.git
%cd /content/catnip

# Authenticate with Google
from google.colab import auth
auth.authenticate_user()

# Install gcsfuse
!echo "deb http://packages.cloud.google.com/apt gcsfuse-bionic main" > /etc/apt/sources.list.d/gcsfuse.list
!curl https://packages.cloud.google.com/apt/doc/apt-key.gpg | apt-key add -
!apt -qq update
!apt -qq install gcsfuse

# Mount bucket
!mkdir -p /content/catnip/data
!gcsfuse --implicit-dirs catnip-data /content/catnip/data

# Install packages not included in Colab
%pip install ultralytics pydantic pydantic-settings omegaconf

import os
from pathlib import Path

### Local

In [1]:
# Autoreload for debugging
%load_ext autoreload
%autoreload 2

import os
from pathlib import Path
os.chdir(Path.cwd().parent)

### General

In [2]:
# Dependencies and configuration
from src.config import settings, setup_dirs

print("Working directory set to:", Path.cwd())
izutsumiPaths, notIzutsumiPaths = setup_dirs()

# For Apple Silicon
%env PYTORCH_ENABLE_MPS_FALLBACK=1

Working directory set to: /Users/rifusaki/repos/catnip
env: PYTORCH_ENABLE_MPS_FALLBACK=1


## Pre-processing

### Panel extraction

In [ ]:
from modules.coreMPE.src.adenzu_panel.image_processing import panel


_ = panel.extract_panels_for_images_in_folder_recursive(
    input_dir=str(settings.paths.pages_dir),
    output_dir=str(settings.paths.panels_dir),
    split_joint_panels=False,   # maps to --split-joint-panels
    fallback=True              # maps to --fallback
)

### Head crops

In [ ]:
from src.preprocess.headExtraction import anime_extraction_recursive


num_crops = anime_extraction_recursive(device='mps')
print(f"Extracted {num_crops} faces")

Detecting faces in panels:   0%|          | 12/15949 [00:08<3:02:43,  1.45it/s]


KeyboardInterrupt: 

## Catnip core

### Dataset preparation

In [ ]:
from src.training.preparation import prepare_data

prepare_data(izutsumiPaths, notIzutsumiPaths, version=11)

### Training

In [ ]:
from ultralytics import YOLO

model = YOLO(settings.paths.model_dir/'yolo11x_izutsumiV1.pt')
model.info(detailed=False)

In [ ]:
# YOLOv8 data="config/izutsumiTraining11.yaml",
model.train(
        data="data/recognition/izutsumiTraining",
        epochs=200,
        imgsz=settings.params.img_size,
        batch=16,
        lr0=1e-4,       # lower LR for finetuning
        freeze=2,      # freeze backbone layers
        project="runs/train",
        name="v11",
        device='cpu', # "mps", "cuda", "cpu"
        workers=8,
        resume=False,
        cache=False     # True for Colab
    )

### Evaluation

In [ ]:
# Evaluate
metrics = model.val()
print(metrics)  

### Prediction

In [ ]:
# Predict on unseen images
# data/recognition/izutsumiTraining/val/**/*.jpg
results = model.predict(
    source="data/preprocess/crops/**/*.jpg",
    save_txt=True,
    conf=0.5,
    stream=True
)

for r in results:
    next(results)

In [ ]:
import pandas as pd

df = pd.read_csv("predictions.csv")
top = df.sort_values("confidence", ascending=False)

# Show top 10
print(top.head(10))